In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_classifier import DNNModel
from random import randint
from visualizer import *


2023-01-11 19:34:03.102912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-11 19:34:03.103012: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-11 19:34:03.103025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:

data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data)

In [ ]:
fm.show_raw_visualization()

In [3]:
model = DNNModel(neg_cutoff=0.40,pos_cutoff=0.6,epochs=50,train_size=0.7,val_size=0.15)
model.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
model.prepare_data(data = fm.df,cols = fm.cols,random_state=1)
accurracy, coverage = model.run(gpu = False)

2023-01-11 19:34:08.974762: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-01-11 19:34:08.974816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: andy-GA-970A-D3
2023-01-11 19:34:08.974831: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: andy-GA-970A-D3
2023-01-11 19:34:08.974994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 520.61.5
2023-01-11 19:34:08.975052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2023-01-11 19:34:08.975069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 520.61.5


Train = 1266, Val = 271, Test = 272, All = 1809
Epoch 1/50

Epoch 1: val_loss improved from inf to 0.70167, saving model to model_dnn_checkpoint.h5
40/40 - 2s - loss: 0.7486 - accuracy: 0.5142 - val_loss: 0.7017 - val_accuracy: 0.5055 - 2s/epoch - 43ms/step
Epoch 2/50

Epoch 2: val_loss improved from 0.70167 to 0.70100, saving model to model_dnn_checkpoint.h5
40/40 - 0s - loss: 0.7432 - accuracy: 0.5071 - val_loss: 0.7010 - val_accuracy: 0.4982 - 227ms/epoch - 6ms/step
Epoch 3/50

Epoch 3: val_loss did not improve from 0.70100
40/40 - 0s - loss: 0.7329 - accuracy: 0.4937 - val_loss: 0.7016 - val_accuracy: 0.4834 - 198ms/epoch - 5ms/step
Epoch 4/50

Epoch 4: val_loss improved from 0.70100 to 0.70096, saving model to model_dnn_checkpoint.h5
40/40 - 0s - loss: 0.7352 - accuracy: 0.4953 - val_loss: 0.7010 - val_accuracy: 0.4982 - 249ms/epoch - 6ms/step
Epoch 5/50

Epoch 5: val_loss did not improve from 0.70096
40/40 - 0s - loss: 0.7154 - accuracy: 0.5174 - val_loss: 0.7020 - val_accuracy: 

In [ ]:
model.analyze_predict_by_cutoff()

In [ ]:
model.visualize_efficiency_by_cutoff(min_delta=0,max_delta=0.17)

In [ ]:
model.predict_with_cutoff(neg_cutoff = 0.5 - 0.14,pos_cutoff = 0.5 + 0.14)

In [ ]:
acc_list = []
cov_list = []
laps = 5

for i in range (0,laps):
    print("\n======= Lap {} =======".format(i+1))
    model = DNNModel(neg_cutoff=0.40,pos_cutoff=0.6,epochs=50,train_size=0.7,val_size=0.15)
    model.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
    model.prepare_data(data = fm.df,cols = fm.cols,random_state=i)
    accurracy, coverage = model.run(gpu = False)
    model.visualize_loss()
    model.visualize_accuracy()
    acc_list.append(accurracy)
    cov_list.append(coverage)

acc_arr = np.array(acc_list)
cov_arr = np.array(cov_list)

print("\n======")
print("Accuracy mean: {}, std: {}".format(acc_arr.mean(),acc_arr.std()))
print("Coverage mean: {}, std: {}".format(cov_arr.mean(),cov_arr.std()))
print("======")